<a href="https://colab.research.google.com/github/sitiaisyah14/Machine-Learning_2022/blob/main/UTS_Bagian_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proses Ekstrasi Fitur, Pembuatan Model dan Evaluasi dengan Naïve Bayes**

---

Proses dimulai dari 
*   Proses pengolahan/penghapusan tanda mention @ pada dataset
*   Ekstraksi Fitur dengan Bag of Words (Count)
*   Pembuatan Model
*   Evaluasi


In [ ]:
##Perintah untuk melakukan import file yang berisi dataset

from google.colab import files
uploaded = files.upload()

Saving tweet_emotions.csv to tweet_emotions.csv


**Proses import Load Dataset dan membaca dataset dengan Pandas**

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('tweet_emotions.csv', encoding='latin-1')
 # spesifiksi encoding diperlukan karena data tidak menggunakan UTF-8

df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


**Proses Penghapusan Kolom "tweet_id"**

In [ ]:
# Drop kolom tweet_id
df.drop('tweet_id', axis=1, inplace=True)
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


**Inspeksi Data**

In [ ]:
# Inspeksi data
print(df['sentiment'].value_counts())
print('\n')

print(df.info())
print('\n')

print(df.describe())

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  40000 non-null  object
 1   content    40000 non-null  object
dtypes: object(2)
memory usage: 625.1+ KB
None


       sentiment                                            content
count      40000                                              40000
unique        13                                              39827
top      neutral  I just received a mothers day card from my lov...
freq        8638                                                 14


**Penanganan Mention @ untuk dihapus**

In [ ]:
#Penanganan Mention @ untuk dihapus 

df["ContentNoPunctuation"] = df['content'].str.replace('[^\w\s]','')
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


,sentiment,content,ContentNoPunctuation
0,empty,@tiffanylue i know i was listenin to bad habi...,tiffanylue i know i was listenin to bad habit...
1,sadness,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhhwaitin on y...
2,sadness,Funeral ceremony...gloomy friday...,Funeral ceremonygloomy friday
3,enthusiasm,wants to hang out with friends SOON!,wants to hang out with friends SOON
4,neutral,@dannycastillo We want to trade with someone w...,dannycastillo We want to trade with someone wh...


**Memisahkan Fitur dengan Label**

In [ ]:
# Memisahkan Fitur dengan Label

X = df['ContentNoPunctuation'].values
y = df['sentiment'].values

**Encode Label pada Label kolom "Sentiment"**

In [ ]:
# Encode Label pada Sentiment
new_labels = {
    'anger': int(1),    
    'boredom': int(2),
    'empty': int(3),
    'enthusiasm': int(4),
    'fun': int(5),
    'happiness': int(6),
    'hate': int(7),
    'love': int(8),
    'neutral': int(9),
    'relief': int(10),
    'sadness': int(11),
    'suprise': int(12),
    'wory': int(13)
}

# Encode label
df['sentiment'] = df['sentiment'].map(new_labels)
df.head()

,sentiment,content,ContentNoPunctuation
0,3.0,@tiffanylue i know i was listenin to bad habi...,tiffanylue i know i was listenin to bad habit...
1,11.0,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhhwaitin on y...
2,11.0,Funeral ceremony...gloomy friday...,Funeral ceremonygloomy friday
3,4.0,wants to hang out with friends SOON!,wants to hang out with friends SOON
4,9.0,@dannycastillo We want to trade with someone w...,dannycastillo We want to trade with someone wh...


**Proses Ekstraksi fitur dengan menggunakan CountVectorizer**

In [ ]:
#Ekstraksi fitur dengan menggunakan CountVectorizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Split data training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

# Inisiasi CountVectorizer
bow = CountVectorizer(stop_words='english')

# Fitting dan transform X_train dengan CountVectorizer
X_train = bow.fit_transform(X_train)

# Transform X_test
# Mengapa hanya transform? Alasan yang sama dengan kasus pada percobaan ke-3
# Kita tidak menginginkan model mengetahui paramter yang digunakan oleh CountVectorizer untuk fitting data X_train
# Sehingga, data testing dapat tetap menjadi data yang asing bagi model nantinya
X_test = bow.transform(X_test)

**Menghitung nilai X_Train**

In [ ]:
print(X_train.shape)

(32000, 45374)


**Training Pembuatan Model dan Evaluasi**

In [ ]:
#Training Pembuatan Model dan Evaluasi

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Inisiasi MultinomialNB
mnb = MultinomialNB()

# Fit model
mnb.fit(X_train, y_train)

# Prediksi dengan data training
y_pred_train = mnb.predict(X_train)

# Evaluasi akurasi data training
acc_train = accuracy_score(y_train, y_pred_train)

# Prediksi dengan data training
y_pred_test = mnb.predict(X_test)

# Evaluasi akurasi data training
acc_test = accuracy_score(y_test, y_pred_test)

# Print hasil evaluasi
print(f'Hasil akurasi data train: {acc_train}')
print(f'Hasil akurasi data test: {acc_test}')

Hasil akurasi data train: 0.61409375
Hasil akurasi data test: 0.319875
